# Conformal Geometry

Conformal geometry studies *conformal structures*, or, *conformal trasnformations*.
Topology is concerned with preserving connectivity (e.g. number of holes), while
conformal geometry is concerned with preserving angles.
Note that "conformity" is a stronger notion than topology, in the sense
that a conformal equivalence implies topological equivalence.

When modelling geometric objects, we can deal with different levels of abstraction. For example,
if we want to create a type "Triangle" that encapsulates all triangles, what is the "correct" equivalence
class to use for such modelling? We know that, from the start, a triangle has three vertices with v1
connected to v2, v2 connected to v3 and v1. From an topological perspective, this view is yet lacking. We don't
know if the triangle is filled or not, hence, it could either have one hole or zero holes. For our case,
let's suppose that a triangle is filled, i.e. it's a simpicial complex of $[v1,v2,v3,[v1,v2], [v2,v3],[v3,v1],[v1,v2,v3]]$.
Once this is defined, then we've characterized triangles topologically. Yet, note that, topologically, a triangle is the same
as a filled circle. Hence, the topology does not actually capture the "shape" of the triangle. This is done with
conformal equivalence.

Hence, besides the simplicial complex description (which is the algebraic topology), we need an algebraic conformal description.
This will be done using Geometric Algebra with the Conformal Model.


## 1. Conformal Algebra

Our goal is to model 3D space in a way that conformal transformations can be algebraically represented. Besides the algebraic
representation, we want our transformations to be covariant, in the sense that applying one conformal transformation preserves
the algebraic structure of our object.

"All conformal transformations are generated by versor products using the elementary
vectors of the conformal model... The fact that the general conformal transformations can be represented as versors ﬁnally
explains the name of the conformal model." (Leo Dorst, Geometric Algebra for Computer Science).

A conformal transformation is a map that locally preserves angles. This is of interest to us! Why?
Because the idea of constructing a diagram is that it's construction describes the shapes, yet,
lengths, locations, colors can change. For example, we want to define what a triangle is by specifying that
it has 3 vertices and 3 faces. Yet, this triangle might be rotated, translated, enlarged, and so on.

In [10]:
using Pkg, Revise
Pkg.activate("../.")

  Activating project at `~/MEGA/EMAP/Diagrams.jl`


In [135]:
using CliffordAlgebras
import CliffordAlgebras: basegrade
using LinearAlgebra: norm, normalize, dot
using Plots

function getgradesdict(cl::CliffordAlgebra)
    gradesdict = Dict(g =>[] for g in 0:sum(signature(cl)))
    ngrades = sum(signature(cl))
    for g in 0:ngrades
        for (i,p) in enumerate(propertynames(cl))
            if i <= sum(binomial.(dimcl,0:g))
                push!(gradesdict[g],p)
            end
        end
    end

    return gradesdict
end

function Base.:≈(a::MultiVector,b::MultiVector)
    vector(a) ≈ vector(b)
end

function multivector(cl::CliffordAlgebra,v::Vector)
    bases = propertynames(cl)
    mapreduce(x->x[1] * getproperty(cl,x[2]),+, zip(v,bases))
end

function Base.map(f, c::MultiVector)
    multivector(algebra(c),map(f, vector(c)))
end

function Base.reduce(op, c::MultiVector)
    reduce(op, vectorize(c))
end

"""
vectorize(c::MultiVector)

Turns a multivector into a vector of multivectors,
where each element in a multivector with a single blade.
"""
function vectorize(c::MultiVector)
    filter(x->!isapprox(norm_sqr(x),0),basevector.(Ref(algebra(c)), propertynames(c)) .* vector(c))
end

"""
unvectorize(c::MultiVector)

Turns a vector of multivectors into a multivector by
summing each component.
"""
function unvectorize(c::MultiVector)
    reduce(+,vectorize(B))
end

"""
basesfromgrade(cl::CliffordAlgebra, k::Int)

Returns a list of base multivector for a given
grade `k`. Note that grades go from ``0`` to ``n``, 
where ``n`` is the dimension, given by the sum of
the signature, e.g. ``R^{3,1,1}`` has
``n = 5``.
"""
function basesfromgrade(cl::CliffordAlgebra, k::Int)
    ngrades = sum(signature(cl))
    i = sum(binomial.(ngrades,0:k-1))+1
    j = sum(binomial.(ngrades,0:k))
    basesymbol.(Ref(cl),i:j)
end


function norm_corr(a::MultiVector)
    sqrt(abs(norm_sqr(a)))
end


bladenormalization(B::MultiVector) = B / norm_corr(B)
orthogonalproj(c::MultiVector, B::MultiVector) = (c ⨼ B) * inv(B)
orthogonalreject(a::MultiVector, B::MultiVector) = (a ∧ B) ⨽ inv(B)

"""
maxgrade(A::MultiVector)

Returns the maximum grade ``k`` of a multivector A
with non-null value, i.e. ``\\langle A \\rangle_{max}``
is equal to `grade(A,maxgrade(A))`.
"""
function maxgrade(A::MultiVector)
    D = sum(signature(algebra(A)))
    i = findlast(!isapprox(0), norm_sqr(grade(A,k)) for k in 0:D)
    if isnothing(i)
        return 0
    end
    return i - 1
end
"""
mingrade(A::MultiVector)

Returns the minimum grade ``k`` of a multivector A
with non-null value, or returns 0.
"""
function mingrade(A::MultiVector)
    D = sum(signature(algebra(A)))
    i = findfirst(!isapprox(0), norm_sqr(grade(A,k)) for k in 0:D)
    if isnothing(i)
        return 0
    end
    return i - 1
end


function norm_corr(a::MultiVector)
    sqrt(abs(norm_sqr(a)))
end

bladenormalization(B::MultiVector) = B / norm_corr(B)
orthogonalproj(c::MultiVector, B::MultiVector) = (c ⨼ B) * inv(B)
orthogonalreject(a::MultiVector, B::MultiVector) = (a ∧ B) ⨽ inv(B)

function findmaxmv(f, a::MultiVector)
    cl = algebra(a)
    scalar, base = findmax(f,vector(a))
    return scalar * basevector(cl, base)
end

function deltaproduct(A::MultiVector,B::MultiVector)
    C = A*B
    return grade(C, maxgrade(C))
end

function span(a::MultiVector)
    if length(coefficients(a)) != 1
        error("This function takes only multivectors with a single coeffient")
    end
    if mingrade(a) ≤ 1
        return [a/norm(a)]
    end
    cl = algebra(a)
    A  = a / norm(a)
    b = []
    for e in basevector.(Ref(cl),basesfromgrade(cl,1))
        if !(norm_sqr(A ⋅ e) ≈ 0)
            push!(b, e)
        end
    end
    return b
end

"""
factorizeblade(B::MultiVector)

It takes a blade as input and factorizes it,
i.e. B = f1 ∧ f2 ∧... ∧fk
"""
function factorizeblade(B::MultiVector)
    s = norm_corr(B)
    temp = bladenormalization(B)
    mask = basevector.(Ref(algebra(B)),basesfromgrade(algebra(B),1))
    k    = mingrade(B)
    nfactors = 1
    factors  = []
    
    for i in nfactors:k-1
        for ej in mask
            proj = orthogonalproj(ej, temp)
            if !isapprox(norm_corr(proj),0.0; atol=1e-10)
                factorj = bladenormalization(proj)
                temp = inv(factorj) ⨼ temp
                push!(factors, factorj)
                break
            end
        end
    end
    factork = bladenormalization(temp)
    push!(factors, factork)
    return s, factors
end

function intersectionunionblades(a::MultiVector, b::MultiVector)
    cl = algebra(a)
    A = a
    B = b
    r = mingrade(a)
    s = mingrade(b)
    if r > s
        A = b
        B = a
    end
    delta = deltaproduct(A,B)
    t = Int((r + s - mingrade(delta))/2)
    s,factors = factorizeblade(dual(delta))
    inter = cl.𝟏
    uniao = pseudoscalar(cl)
    for factor in factors
        proj = orthogonalproj(factor,A)
        if !isapprox(norm_sqr(proj),0)
            inter = inter ∧ proj
            if mingrade(inter) == t
                uniao = (A ⨽ inv(inter)) ∧ B
                break
            end
        end
        
        reje = orthogonalreject(factor,A)
        # reje = factor - proj
        if !isapprox(norm_sqr(reje),0)
            uniao = reje ⨼ uniao
            if mingrade(uniao) == r + s - t
                inter = dual(B) ⨼ A
                break
            end
        end
    end
    
    if r > s
        uniao = uniao * (-1)^((r-t)*(s-t))
        inter = inter * (-1)^((r-t)*(s-t))
    end
    return inter, uniao
end

function meet(a::MultiVector, b::MultiVector)
    meet,_ = intersectionunionblades(a,b)
    return meet
end

function union(a::MultiVector, b::MultiVector)
    _, un = intersectionunionblades(a,b)
    return un
end

union (generic function with 1 method)

## Bases - Metric

In conformal geometry, our representational space is 

In [195]:
cl = CliffordAlgebra(:CGA3D)

𝟏  = cl.𝟏
e1 = cl.e1
e2 = cl.e2
e3 = cl.e3
e₊ = cl.e₊
e₋ = cl.e₋
I  = pseudoscalar(cl)

no = (cl.e₊ + cl.e₋)/2
n∞ = cl.e₋ - cl.e₊

propertynames(cl)

(:𝟏, :e1, :e2, :e3, :e₊, :e₋, :e1e2, :e1e3, :e2e3, :e1e₊, :e2e₊, :e3e₊, :e1e₋, :e2e₋, :e3e₋, :e₊e₋, :e1e2e3, :e1e₊e2, :e1e3e₊, :e2e₊e3, :e1e2e₋, :e1e₋e3, :e2e3e₋, :e1e₊e₋, :e2e₋e₊, :e3e₊e₋, :e1e2e3e₊, :e1e2e₋e3, :e1e2e₊e₋, :e1e3e₋e₊, :e2e3e₊e₋, :e1e2e3e₊e₋)

In [196]:
@show e₊ ⋅ e₋
@show e₊ ⋅ e₊
@show no ⋅ n∞
@show no ⋅ no
@show n∞ ⋅ n∞;

e₊ ⋅ e₋ = 0
e₊ ⋅ e₊ = +1 ∈ Cl(4, 1, 0)
no ⋅ n∞ = -1.0 ∈ Cl(4, 1, 0)
no ⋅ no = 0
n∞ ⋅ n∞ = 0


In [197]:
point(x=0,y=0,z=0) = no + x*cl.e1 + y*cl.e2 + z*cl.e3 + (x^2 + y^2 + z^2)/2 * n∞
point(;x=0,y=0,z=0) = no + x*cl.e1 + y*cl.e2 + z*cl.e3 + (x^2 + y^2 + z^2)/2 * n∞

point (generic function with 4 methods)

In [198]:
a = point()
b = point(1,1)
l1 = bladenormalization(a ∧ b ∧ n∞)

a = point(0,2)
b = point(2,2)
l2 = bladenormalization(a ∧ b ∧ n∞)


x = point(2,2)

+2.0×e1+2.0×e2-3.5×e₊+4.5×e₋ ∈ Cl(4, 1, 0)

In [232]:
n = 5
r = maxgrade(l1)
s = maxgrade(l2)
g = 2n - r - s

+0.7071067811865475-0.7071067811865475×e1e2+1.414213562373095×e1e₊-1.414213562373095×e2e₊-1.414213562373095×e1e₋+1.414213562373095×e2e₋ ∈ Cl(4, 1, 0)

In [237]:
l1*l1

+0.9999999999999998 ∈ Cl(4, 1, 0)

In [181]:
x ∧ (l1 + l2)

0

In [141]:
# no = (-cl.e₊ + cl.e₋)/2
# n∞ = cl.e₋ + cl.e₊

line = bladenormalization(point(2) ∧ point(2,0,1) ∧ n∞)

a = point(0,0,1)
b = point(1,0,1)
c = point(0,1,1)
plane = bladenormalization(a ∧ b ∧ c ∧ n∞)

m = map(x->round(x,digits=10),meet(line,plane))

-4.0×e1e3-2.0×e1e₊+1.0×e3e₊+2.0×e1e₋+1.0×e3e₋+1.0×e₊e₋ ∈ Cl(4, 1, 0)

In [216]:
(I*line) ⨼ plane

-2.0×e1e₊-1.0×e3e₊+2.0×e1e₋+1.0×e3e₋+1.0×e₊e₋ ∈ Cl(4, 1, 0)

In [221]:
# I*((I*line)∧ (I*line)) == dual(dual(plane) ∧ dual(line))

+2.0×e1e₊+1.0×e3e₊-2.0×e1e₋-1.0×e3e₋-1.0×e₊e₋ ∈ Cl(4, 1, 0)

In [158]:
s, factors = factorizeblade(line)
s*reduce(∧,factors) ≈ line

true

true

-2.0×e1e3e₊-2.0×e1e₋e3-1.0×e3e₊e₋ ∈ Cl(4, 1, 0)

In [165]:
dual(dual(line) ∧ dual(plane))

+2.0×e1e₊+1.0×e3e₊-2.0×e1e₋-1.0×e3e₋-1.0×e₊e₋ ∈ Cl(4, 1, 0)

-2.0×e1e3e₊-2.0×e1e₋e3-1.0×e3e₊e₋ ∈ Cl(4, 1, 0)

In [125]:
line

+2.0×e1e3e₊-2.0×e1e₋e3+1.0×e3e₊e₋ ∈ Cl(4, 1, 0)

In [126]:
plane

+1.0×e1e2e3e₊-1.0×e1e2e₋e3-1.0×e1e2e₊e₋ ∈ Cl(4, 1, 0)

In [98]:
# meet(line,plane)

+1.0×e1e2e3e₊-1.0×e1e2e₋e3+1.0×e1e2e₊e₋ ∈ Cl(4, 1, 0)

In [99]:
m

+4.0×e1e3+2.0×e1e₊-1.0×e3e₊+2.0×e1e₋+1.0×e3e₋+1.0×e₊e₋ ∈ Cl(4, 1, 0)

In [39]:
# grade(l1*l2,5
n = sum(signature(cl))
r = mingrade(l1)
s = mingrade(l2)

# grade(l1*l2, 2n - r - s)
l1*l2

+2.0-2.0×e1e2+4.0×e1e₊-4.0×e2e₊-4.0×e1e₋+4.0×e2e₋ ∈ Cl(4, 1, 0)

In [40]:
a ∧ b ∧ n∞

+1.0×e1e3e₊+1.0×e1e₋e3-1.0×e1e₊e₋ ∈ Cl(4, 1, 0)

+2.0×e2-1.5×e₊+2.5×e₋ ∈ Cl(4, 1, 0)

In [9]:
a

+2.0×e2-1.5×e₊+2.5×e₋ ∈ Cl(4, 1, 0)

In [85]:
n∞

-1×e₊+1×e₋ ∈ Cl(4, 1, 0)